# Automatyczne pozyskiwanie danych

## Tomasz Rodak

Wykład 6

---

## asyncio - obsługa błędów i kasowanie zadań

### Obsługa błędów 

Wyjątki w `asyncio` mogą być zgłaszane zarówno w korutynach wywoływanych przez `await`, jak i w zadaniach utworzonych przez `asyncio.create_task()`. Jeśli korutyna zgłosi wyjątek, zostanie on przekazany do miejsca, gdzie wywołano `await`. W przypadku zadań, wyjątek nie zostanie automatycznie zgłoszony, dopóki nie wywołamy na zadaniu `await` lub nie sprawdzimy go przez `task.exception()`.

Przykład obsługi wyjątków w korutynie:

```python
import asyncio

async def risky():
    raise ValueError("Błąd!")

async def main():
    try:
        await risky()
    except ValueError as e:
        print(f"Złapano wyjątek: {e}")

asyncio.run(main())
```

W przypadku zadań:

```python
async def main():
    task = asyncio.create_task(risky())
    try:
        await task
    except ValueError as e:
        print(f"Zadanie zakończone wyjątkiem: {e}")

asyncio.run(main())
```

Jeśli nie wyczekamy zadania, wyjątek może pozostać niezauważony.

### Kasowanie zadań i zarządzanie cyklem życia

#### Metoda `task.cancel()` 

Zadanie asynchroniczne można anulować z zewnątrz za pomocą metody `cancel()`, która powoduje zgłoszenie wyjątku `asyncio.CancelledError` w miejscu, gdzie zadanie wykonuje `await`. 

Wyjątek `asyncio.CancelledError` jest specjalnym wyjątkiem wskazującym, że zadanie zostało anulowane. Można go obsłużyć, aby przeprowadzić czyszczenie zasobów, ale zwykle powinien być przekazany dalej, aby zadanie mogło się prawidłowo zakończyć.

Ważne aspekty:  

- Zadanie anulowane uważane jest za zakończone
- Wyjątek `CancelledError` jest propagowany do miejsca, gdzie wywołano `await` na zadaniu
- Jeśli obsługujemy ten wyjątek, dobrą praktyką jest ponowne jego zgłoszenie po wykonaniu niezbędnych operacji czyszczenia

Przykład:

```python
import asyncio

async def wait_with_countdown(seconds, verbose=True):
    for i in range(seconds, 0, -1):
        if verbose:
            print(f"Zamknięcie za {i} sekund...", end="\r")
        await asyncio.sleep(1)
    print("Zamknięcie...")

async def long_task(seconds, verbose=True):
    try:
        print("Zadanie rozpoczęte")
        await wait_with_countdown(seconds, verbose=verbose)
        print("Zadanie zakończone")
        return "Rezultat"
    except asyncio.CancelledError:
        print("Zadanie zostało anulowane!")
        raise  # Ważne: przekazanie wyjątku dalej
    
async def main():
    task = asyncio.create_task(long_task(10, verbose=True))
    await asyncio.sleep(3)  # Pozwól zadaniu się rozpocząć
    task.cancel()  # Anuluj zadanie
    try:
        await task  # Poczekaj na zakończenie zadania (anulowanie)
    except asyncio.CancelledError:
        print("Główna funkcja obsłużyła anulowanie")

asyncio.run(main())
```

#### Wzorzec czyszczenia zasobów po anulowaniu

Zadania mogą korzystać z zewnętrznych zasobów (połączenia sieciowe, pliki, itp.), które powinny być zwolnione nawet w przypadku anulowania. Korzystamy wtedy z wzorca `try-except-finally`:

```python
async def task_with_resources():
    resource = acquire_resource()  # Pozyskiwanie zasobu
    try:
        await process_with_resource(resource)
    except asyncio.CancelledError:
        print("Zadanie anulowane, czyszczenie...")
        raise  # Propaguj wyjątek
    finally:
        resource.release()  # Zwolnij zasób niezależnie od wyniku
```

W praktyce konstrukcja `async with` często udostępnia ten wzorzec automatycznie, np. dla połączeń sieciowych czy plików. Niżej przykłady dla `aiohttp`.

#### Metody obiektu Task - zarządzanie zadaniami

Klasa `asyncio.Task` dostarcza wielu metod ułatwiających obsługę zadań asynchronicznych:

- `task.cancel()` - anuluje zadanie, zgłaszając wyjątek `CancelledError` przy najbliższym `await`
- `task.done()` - zwraca `True`, jeśli zadanie zostało zakończone (normalnie, przez wyjątek lub anulowanie)
- `task.cancelled()` - zwraca `True`, jeśli zadanie zostało anulowane
- `task.exception()` - zwraca wyjątek, jeśli zadanie zakończyło się błędem, `None` jeśli zakończyło się prawidłowo, lub rzuca wyjątek `InvalidStateError` jeśli zadanie jeszcze nie zakończyło działania
- `task.result()` - zwraca wynik zadania lub rzuca wyjątek, jeśli zakończyło się błędem; rzuca `InvalidStateError` jeśli zadanie nie zakończyło jeszcze działania
- `task.get_name()` / `task.set_name()` - uzyskuje/ustawia nazwę zadania (przydatne do debugowania)
- `task.get_coro()` - zwraca korutynę, która została przekazana do zadania

Przykład monitorowania stanu zadania:

```python
import asyncio
import time

async def background_task(name):
    try:
        for i in range(10):
            print(f"Task {name}: step {i}")
            await asyncio.sleep(0.5)
        return f"Task {name} completed successfully"
    except asyncio.CancelledError:
        print(f"Task {name} was cancelled")
        raise

async def monitor_task(task):
    while not task.done():
        print(f"Task status: done={task.done()}, cancelled={task.cancelled()}")
        await asyncio.sleep(1)
    
    try:
        result = task.result()
        print(f"Task result: {result}")
    except asyncio.CancelledError:
        print("Task was cancelled (from result)")
    except Exception as e:
        print(f"Task raised an exception: {e}")

async def main():
    # Utwórz i nazwij zadanie
    task = asyncio.create_task(background_task("A"))
    task.set_name("background_demo")
    
    # Uruchom monitoring zadania
    monitor = asyncio.create_task(monitor_task(task))
    
    # Poczekaj 2 sekundy i anuluj zadanie
    await asyncio.sleep(2)
    print(f"Anulowanie zadania {task.get_name()}")
    task.cancel()
    
    # Poczekaj na zakończenie monitorowania
    await monitor

# asyncio.run(main())
```

### Timeouty i oczekiwanie na wyniki

Narzędzia do obsługi timeoutów i efektywnego oczekiwania na zakończenie zadań.

#### `asyncio.wait_for()` - zarządzanie timeoutami

`asyncio.wait_for()` pozwala na ustawienie maksymalnego czasu oczekiwania na zakończenie zadania. Jeśli zadanie nie zakończy się w określonym czasie, zgłaszany jest wyjątek `asyncio.TimeoutError`. Przykład użycia:

```python
async def main():
    try:
        await asyncio.wait_for(long_task(10), timeout=5.0)
    except asyncio.TimeoutError:
        print("Zadanie przekroczyło dozwolony czas wykonania")

asyncio.run(main())
```

#### `asyncio.as_completed()` 

Funkcja `asyncio.as_completed()` zwraca iterator, który zwraca zakończone zadania w miarę ich dostępności.

Przykład:

```python
import asyncio
import random

async def random_wait(name):
    wait_time = random.randint(1, 5)
    await asyncio.sleep(wait_time)
    return f"{name} zakończone po {wait_time} sekundach"
    
async def main():
    tasks = []
    for i in range(5):
        task = asyncio.create_task(random_wait(f"Zadanie {i+1}"))
        tasks.append(task)
    
    for completed_task in asyncio.as_completed(tasks):
        result = await completed_task
        print(result)
    print("Wszystkie zadania zakończone")

asyncio.run(main())
```

#### `asyncio.gather()` 

Funkcja `asyncio.gather()` pozwala na równoległe wykonanie wielu korutyn i zebranie ich wyników. Parametr `return_exceptions` kontroluje, jak obsługiwane są wyjątki:

- Gdy `return_exceptions=False` (domyślnie): jeśli którakolwiek korutyna zgłosi wyjątek, zostanie on natychmiast zgłoszony z `gather()`, a pozostałe korutyny będą kontynuować działanie w tle.

- Gdy `return_exceptions=True`: wyjątki są zbierane jako zwykłe wyniki i zwracane w wynikowej liście, co pozwala na dostarczenie częściowych wyników nawet gdy niektóre zadania zakończyły się niepowodzeniem.

Przykład obsługi wyjątków z `gather()`:

```python
import asyncio
import random

async def risky_task(task_id):
    await asyncio.sleep(random.uniform(0.5, 2))
    if random.random() < 0.5:  # 50% szans na błąd
        raise ValueError(f"Błąd w zadaniu {task_id}")
    return f"Wynik z zadania {task_id}"

async def with_return_exceptions():
    print("\nUżycie return_exceptions=True:")
    # Wyjątki są zwracane jako część wyników
    results = await asyncio.gather(
        *[risky_task(i) for i in range(5)],
        return_exceptions=True
    )
    
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Zadanie {i} zgłosiło wyjątek: {result}")
        else:
            print(f"Zadanie {i} zwróciło: {result}")

async def without_return_exceptions():
    print("\nUżycie return_exceptions=False (domyślne):")
    try:
        # Pierwszy wyjątek natychmiast przerwie wykonanie gather()
        results = await asyncio.gather(
            *[risky_task(i) for i in range(5)]
        )
        
        print("Wszystkie zadania zakończone sukcesem:")
        for i, result in enumerate(results):
            print(f"Zadanie {i}: {result}")
    except ValueError as e:
        print(f"Przerwano z powodu wyjątku: {e}")

async def main():
    await with_return_exceptions()
    await without_return_exceptions()

asyncio.run(main())
```

## aiohttp

Biblioteka do wykonywania asynchronicznych zapytań HTTP w Pythonie. Pozwala na obsługę wielu żądań równolegle, co jest niemożliwe w tradycyjnych, synchronicznych bibliotekach takich jak `requests`. Instalacja: `pip install aiohttp`.

Główne komponenty to `ClientSession` (zarządza połączeniami) i `ClientResponse` (reprezentuje odpowiedź serwera).

### Klient HTTP i podstawowe operacje

Podstawowy sposób użycia polega na utworzeniu sesji i wykonaniu zapytania GET:

```python
import aiohttp
import asyncio

async def main():
    async with aiohttp.ClientSession() as session:
        async with session.get('https://httpbin.org/get') as response:
            print(response.status)
            data = await response.text()
            print(data)

asyncio.run(main())
```

Można przekazywać parametry URL, nagłówki i korzystać z kontekstu asynchronicznego (`async with`).

### Zarządzanie sesją

Sesja (`ClientSession`) powinna być współdzielona między wieloma zapytaniami, aby zoptymalizować wykorzystanie połączeń. Pozwala to także na automatyczne zarządzanie cookies i utrzymywanie stanu. Po zakończeniu pracy sesję należy zamknąć (najlepiej stosować `async with`).

Można ustawiać timeouty, limity połączeń i inne opcje konfiguracyjne:

```python
import aiohttp
import asyncio

async def main():
    timeout = aiohttp.ClientTimeout(total=5)
    async with aiohttp.ClientSession(timeout=timeout) as session:
        ... # zapytania
```

### Obsługa odpowiedzi

Odpowiedź serwera (`ClientResponse`) pozwala odczytać status, nagłówki i treść. Można pobrać dane jako tekst (`await response.text()`), JSON (`await response.json()`) lub binarnie (`await response.read()`).

Przykład:

```python
async with session.get(url) as response:
    if response.status == 200:
        dane = await response.json()
        print(dane)
    else:
        print(f"Błąd: {response.status}")
```

Dla dużych odpowiedzi można czytać dane fragmentami (streaming).

### Obsługa błędów i ponowne próby

Biblioteka aiohttp definiuje własną hierarchię wyjątków, np. `aiohttp.ClientError`. Błędy sieciowe, timeouty czy nieprawidłowe odpowiedzi należy obsługiwać przez `try/except`.

Przykład z ponawianiem prób:

```python
import aiohttp
import asyncio

async def fetch_with_retry(session, url, retries=3):
    for attempt in range(retries):
        try:
            async with session.get(url) as response:
                return await response.text()
        except aiohttp.ClientError as e:
            print(f"Błąd: {e}, próba {attempt+1}")
            await asyncio.sleep(2 ** attempt)  # exponential backoff
    return None
```

## Przykłady integracji asyncio i aiohttp

### Pobieranie wielu zasobów równolegle


```python
import aiohttp
import asyncio

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    urls = [
        'https://httpbin.org/delay/2',
        'https://httpbin.org/delay/3',
        'https://httpbin.org/delay/1'
    ]
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
        for i, result in enumerate(results):
            print(f"Strona {i+1}: {len(result)} znaków")

asyncio.run(main())
```

Liczbę równoległych zapytań można ograniczać przez semafory lub kolejki.